In [5]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit.circuit import Parameter

# Fix font warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Liberation Sans', 'Arial', 'sans-serif']
plt.rcParams['mathtext.default'] = 'regular'

def create_single_ring_layer(qc, beta_param):
    """Add a single layer of ring XY interactions to the circuit."""
    
    # XY interaction (0,1) - First edge of ring
    qc.ry(np.pi/2, 0)       # Rotate qubit 0 to X-Y plane
    qc.ry(np.pi/2, 1)       # Rotate qubit 1 to X-Y plane
    qc.cx(0, 1)             # First CNOT
    qc.rz(beta_param, 1)    # Z-rotation with beta parameter (note: not 2*beta here)
    qc.cx(0, 1)             # Second CNOT
    qc.ry(-np.pi/2, 0)      # Rotate qubit 0 back
    qc.ry(-np.pi/2, 1)      # Rotate qubit 1 back
    qc.barrier()
    
    # XY interaction (1,2) - Second edge of ring
    qc.ry(np.pi/2, 1)       # Rotate qubit 1 to X-Y plane
    qc.ry(np.pi/2, 2)       # Rotate qubit 2 to X-Y plane
    qc.cx(1, 2)             # First CNOT
    qc.rz(beta_param, 2)    # Z-rotation with beta parameter
    qc.cx(1, 2)             # Second CNOT
    qc.ry(-np.pi/2, 1)      # Rotate qubit 1 back
    qc.ry(-np.pi/2, 2)      # Rotate qubit 2 back
    qc.barrier()
    
    # XY interaction (2,0) - Third edge closes the ring
    qc.ry(np.pi/2, 2)       # Rotate qubit 2 to X-Y plane
    qc.ry(np.pi/2, 0)       # Rotate qubit 0 to X-Y plane
    qc.cx(2, 0)             # First CNOT
    qc.rz(beta_param, 0)    # Z-rotation with beta parameter
    qc.cx(2, 0)             # Second CNOT
    qc.ry(-np.pi/2, 2)      # Rotate qubit 2 back
    qc.ry(-np.pi/2, 0)      # Rotate qubit 0 back

def create_3qubit_ring_xy_mixer_trotterized(beta=np.pi/8, trotter_steps=2):
    """Create the 3-qubit ring XY mixer circuit with Trotterization."""
    
    qc = QuantumCircuit(3, name=f'3Q_Ring_XY_Mixer_Trotter{trotter_steps}')
    
    # Create symbolic parameter for display
    beta_param = Parameter('β')
    
    # Each Trotter step uses β/trotter_steps
    beta_per_step = 2 * beta_param / trotter_steps  # 2* because RZ gate needs 2*beta for XY
    
    # Apply trotter_steps repetitions of the ring XY mixer
    for step in range(trotter_steps):
        
        if step > 0:
            qc.barrier(label=f'Trotter Step {step+1}')
        
        # Apply one full ring of XY interactions
        create_single_ring_layer(qc, beta_per_step)
        
        # Add barrier after each Trotter step (except the last)
        #if step < trotter_steps - 1:
         #   qc.barrier()
    
    # For simulation, we need to bind the parameter to actual values
    try:
        # Try assign_parameters method
        qc_bound = qc.assign_parameters({beta_param: beta})
    except AttributeError:
        # Fallback: create a separate numerical circuit
        qc_bound = QuantumCircuit(3, name=f'3Q_Ring_XY_Mixer_Trotter{trotter_steps}_Bound')
        
        # Each Trotter step uses beta/trotter_steps
        beta_per_step_num = 2 * beta / trotter_steps
        
        for step in range(trotter_steps):
            if step > 0:
                qc_bound.barrier()
            
            # XY interaction (0,1)
            qc_bound.ry(np.pi/2, 0)
            qc_bound.ry(np.pi/2, 1)
            qc_bound.cx(0, 1)
            qc_bound.rz(beta_per_step_num, 1)  # Use numerical value
            qc_bound.cx(0, 1)
            qc_bound.ry(-np.pi/2, 0)
            qc_bound.ry(-np.pi/2, 1)
            
            # XY interaction (1,2)
            qc_bound.ry(np.pi/2, 1)
            qc_bound.ry(np.pi/2, 2)
            qc_bound.cx(1, 2)
            qc_bound.rz(beta_per_step_num, 2)  # Use numerical value
            qc_bound.cx(1, 2)
            qc_bound.ry(-np.pi/2, 1)
            qc_bound.ry(-np.pi/2, 2)
            
            # XY interaction (2,0)
            qc_bound.ry(np.pi/2, 2)
            qc_bound.ry(np.pi/2, 0)
            qc_bound.cx(2, 0)
            qc_bound.rz(beta_per_step_num, 0)  # Use numerical value
            qc_bound.cx(2, 0)
            qc_bound.ry(-np.pi/2, 2)
            qc_bound.ry(-np.pi/2, 0)
            
            if step < trotter_steps - 1:
                qc_bound.barrier()
    
    return qc, qc_bound

def plot_3qubit_ring_xy_mixer_trotterized():
    """Plot the 3-qubit ring XY mixer circuit with Trotterization."""
    
    qc_symbolic, qc_bound = create_3qubit_ring_xy_mixer_trotterized(trotter_steps=2)
    
    # Use symbolic circuit for display (shows β/2)
    try:
        # Method 1: Direct matplotlib circuit drawing
        fig, ax = plt.subplots(figsize=(20, 6))
        qc_symbolic.draw(output='mpl', ax=ax, style='default', fold=None, scale=0.8)
        
        plt.tight_layout()
        plt.savefig('3qubit_ring_xy_mixer_trotterized.png', dpi=600, bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        
        print("✓ Saved circuit as '3qubit_ring_xy_mixer_trotterized.png' (Method 1)")
        plt.show()
        
    except Exception as e:
        print(f"Method 1 failed: {e}")
        
        # Method 2: Use circuit_drawer but handle differently
        try:
            plt.figure(figsize=(20, 6))
            fig = circuit_drawer(qc_symbolic, output='mpl', style='default', fold=None, scale=0.8)
            
            if fig is not None:
                plt.savefig('3qubit_ring_xy_mixer_trotterized.png', dpi=600, bbox_inches='tight', 
                            facecolor='white', edgecolor='none')
                print("✓ Saved circuit as '3qubit_ring_xy_mixer_trotterized.png' (Method 2)")
                plt.show()
            else:
                print("Circuit drawer returned None")
                
        except Exception as e2:
            print(f"Method 2 also failed: {e2}")
            print("Falling back to text-only representation")
    
    # Always show text representation
    print("\nCircuit text representation:")
    print(qc_symbolic.draw(output='text'))
    
    return qc_symbolic, qc_bound

def demonstrate_trotterized_xy_effect():
    """Demonstrate what the trotterized 3-qubit ring XY mixer does."""
    
    try:
        # Try newer Qiskit first
        from qiskit_aer import Aer
        from qiskit import transpile
        use_aer = True
    except ImportError:
        try:
            # Try older Qiskit
            from qiskit import Aer
            from qiskit import transpile
            use_aer = True
        except ImportError:
            # Fall back to basic quantum_info
            from qiskit.quantum_info import Statevector
            use_aer = False
    
    print("\n3-Qubit Ring XY Mixer - Trotterized Effect:")
    print("="*55)
    print("Using β = π/4 total (β/2 per Trotter step)")
    print("Comparing: 1 step vs 2 Trotter steps")
    print()
    
    # Test state
    initial_state = '001'
    
    for trotter_steps in [1, 2]:
        print(f"\n--- {trotter_steps} Trotter Step{'s' if trotter_steps > 1 else ''} ---")
        
        # Create circuit with initial state
        qc = QuantumCircuit(3)
        
        # Prepare initial state |001⟩
        qc.x(0)  # Set qubit 0 to |1⟩
        
        # Add trotterized XY mixer
        _, xy_mixer_bound = create_3qubit_ring_xy_mixer_trotterized(
            beta=np.pi/4, trotter_steps=trotter_steps
        )
        qc = qc.compose(xy_mixer_bound)
        
        if use_aer:
            # Use Aer backend
            backend = Aer.get_backend('statevector_simulator')
            transpiled_qc = transpile(qc, backend)
            job = backend.run(transpiled_qc)
            result = job.result()
            statevector = result.get_statevector()
        else:
            # Use quantum_info Statevector
            statevector = Statevector.from_instruction(qc)
        
        print(f"Initial state |{initial_state}⟩ after {trotter_steps} Trotter step{'s' if trotter_steps > 1 else ''}:")
        
        # Display final state probabilities
        probs = np.abs(statevector)**2
        states = ['|000⟩', '|001⟩', '|010⟩', '|011⟩', '|100⟩', '|101⟩', '|110⟩', '|111⟩']
        
        for i, (state, prob) in enumerate(zip(states, probs)):
            if prob > 0.001:
                print(f"  {state}: {prob:.3f}")

def explain_trotterization():
    """Explain the concept and benefits of Trotterization."""
    
    print("\nTROTTERIZATION CONCEPT:")
    print("="*40)
    print("Mathematical Foundation:")
    print("  Instead of: exp(-iβH)")
    print("  We use:     [exp(-iβH/n)]^n  (n = Trotter steps)")
    print()
    print("For 2 Trotter steps:")
    print("  exp(-iβH) ≈ exp(-iβH/2) × exp(-iβH/2)")
    print()
    print("Benefits:")
    print("  ✓ Better approximation of continuous-time evolution")
    print("  ✓ Reduced gate errors (smaller rotation angles)")
    print("  ✓ More fine-grained control over evolution")
    print("  ✓ Can capture intermediate dynamics")
    print()
    print("Trade-offs:")
    print("  ➕ Pro: More accurate quantum simulation")
    print("  ➕ Pro: Smaller individual gate rotations")
    print("  ➖ Con: Longer circuit (more gates)")
    print("  ➖ Con: More noise accumulation")
    print()
    print("Parameter scaling:")
    print("  • 1 step: Each RZ gate uses 2β")
    print("  • 2 steps: Each RZ gate uses β (total still 2β)")
    print("  • n steps: Each RZ gate uses 2β/n")

def main():
    """Generate and demonstrate the trotterized 3-qubit ring XY mixer."""
    
    print("3-QUBIT RING XY MIXER - TROTTERIZED (2 STEPS)")
    print("="*55)
    print("Creating trotterized 3-qubit ring XY mixer...")
    print("Implements: [exp(-iβH_XY/2)]² ≈ exp(-iβH_XY)")
    print("where H_XY = -[(X₀X₁ + Y₀Y₁) + (X₁X₂ + Y₁Y₂) + (X₂X₀ + Y₂Y₀)]")
    print("Ring topology with 2 Trotter steps")
    print()
    
    # Plot the circuit
    qc_symbolic, qc_bound = plot_3qubit_ring_xy_mixer_trotterized()
    
    # Demonstrate the effect
    demonstrate_trotterized_xy_effect()
    
    # Explain Trotterization
    explain_trotterization()
    
    print("\n" + "="*55)
    print("TROTTERIZED RING QAOA MIXER CONCEPT:")
    print("✓ 2 repetitions of the full ring XY pattern")
    print("✓ Each repetition uses β/2 (total evolution = β)")
    print("✓ Better approximation of continuous-time evolution")
    print("✓ More fine-grained control over mixing dynamics")
    print("✓ Useful for adiabatic quantum computing and QAOA")
    print("✓ Trade-off: Longer circuit but potentially better fidelity")

if __name__ == "__main__":
    main()